## データ前処理

In [1]:
import numpy as np
import pandas as pd
import re
import time
import random

In [2]:
dataset=pd.read_csv('sample_data_city.csv')
dataset.iloc[20183]

物件名      ＪＲ山手線 恵比寿駅 14階建 築3年
カテゴリー                賃貸マンション
住所                東京都渋谷区恵比寿１
最寄駅0          ＪＲ山手線/恵比寿駅 歩4分
最寄駅1      東京メトロ日比谷線/広尾駅 歩15分
最寄駅2         東急東横線/代官山駅 歩16分
築年数                      築3年
構造                      14階建
階数                       12階
賃料                      55万円
管理費                   20000円
敷金                     605万円
礼金                         -
間取り                     2LDK
面積                   65.83m2
Name: 20183, dtype: object

In [33]:
def preprocessing(df):
    df_new=df.copy()
    
    wards=[]
    citys=[]
    lines=[]
    stations=[]
    distances=[]
    results=[]
    ages=[]
    builds=[]
    Floors=[]
    prices=[]
    kanrihis=[]
    sikikins=[]
    reikins=[]
    areas=[]
    
    for i in range(0,len(df_new)):
        ###住所を'23区'、'都市名'に分ける
        pattern='.*区'
        repatter = re.compile(pattern)
        result = repatter.match(df['住所'][i])        
        wards.append(result.group()[3:])
        citys.append(df['住所'][i].split('区')[1])
    
        ###最寄駅は最も近い駅名、路線と時間を採用する
#         nearest_station=df['最寄駅0'][i]
#         lines.append(nearest_station.split('/')[0])
#         stations.append(nearest_station.split('/')[1].split(' ')[0])
#         distances.append(int(nearest_station.split(' ')[1][1]))
        
        access=dataset['最寄駅0'][i]
    
        #路線
        line=access.split('/')[0]
    #     print(line)

        #駅と徒歩
        station_distance=access.split('/')[1]
#         print(station_distance)

        #駅
        station=re.findall(r'.*駅', station_distance)
#         print(station)

        #徒歩
        distance=re.findall(r'\d+',access)
    #     print(int(distance[0]))

        lines.append(line)
        if len(station)==1:
            stations.append(station[0])
        else:
            stations.append(station_distance.split(' ')[0])
#             print('================================================================')
            
        distances.append(int(distance[0]))

        
        ###築年数はint型に変換
        age=dataset['築年数']
        if age[i][0]=='新':
            ages.append(0)
        else:   
            ages.append(int(age[i].split('年')[0][1:]))
        
        ###構造
        m=re.findall(r'\d+', dataset['構造'][i])
        if len(m)==1:
            builds.append(int(m[0]))
        elif len(m)==2:
            builds.append(int(m[1]))
        else:
            builds.append(1)

        ###部屋の階数
        floor=dataset['階数'][i]
        f=re.findall(r'\d+',floor)

        if len(f)==1:
            Floors.append(int(f[0]))
        else:
            Floors.append(random.randint(1,int(builds[i])))
        
        ###賃料
        price=dataset['賃料']
        prices.append(float(price[i].split('万')[0])*10000)
        
        ###管理費
        kanrihi=dataset['管理費']
        if kanrihi[i]!='-':
            kanrihis.append(int(kanrihi[i].split('円')[0]))
        else:
            kanrihis.append(float(0))
        
        ###敷金
        sikikin=dataset['敷金']
        if sikikin[i]!='-':
            sikikins.append(float(sikikin[i].split('万')[0])*10000)
        else:
            sikikins.append(float(0))
                
        ###礼金
        reikin=dataset['礼金']
        if reikin[i]!='-':
            reikins.append(float(reikin[i].split('万')[0])*10000)
        else:
            reikins.append(float(0))
        
        ###面積
        area=dataset['面積']
        areas.append(float(area[i].split('m')[0]))
        
        
    ###変換したカラムの追加
    df_new['23区']=wards
    df_new['都市名']=citys
    df_new['路線']=lines
    df_new['最寄駅']=stations
    df_new['徒歩']=distances
    df_new['築年数']=ages
    df_new['構造']=builds
    df_new['階数']=Floors
    df_new['賃料']=prices
    df_new['管理費']=kanrihis
    df_new['敷金']=sikikins
    df_new['礼金']=reikins
    df_new['面積']=areas
    
#     ###カテゴリカル変数に変換
#     df_new['カテゴリー']=df['カテゴリー'].astype('category') 
#     df_new['間取り']=df_new['間取り'].astype('category')
#     df_new['23区']=df_new['23区'].astype('category')
#     df_new['都市名']=df_new['都市名'].astype('category')
#     df_new['路線']=df_new['路線'].astype('category')
#     df_new['最寄駅']=df_new['最寄駅'].astype('category')

    ###不要なカラムを削除
    df_new=df_new.drop('住所',axis=1)
    df_new=df_new.drop(['最寄駅0', '最寄駅1', '最寄駅2'],axis=1)    
    
    return df_new

df_new=preprocessing(dataset)
df_new.head()

,物件名,カテゴリー,築年数,構造,階数,賃料,管理費,敷金,礼金,間取り,面積,23区,都市名,路線,最寄駅,徒歩
0,ＪＲ山手線 大崎駅 3階建 築45年,賃貸マンション,45,3,2,109000.0,0.0,109000.0,109000.0,11LDK,40.00,品川区,北品川５,ＪＲ山手線,大崎駅,10
1,ステージグランデ神楽坂,賃貸マンション,12,8,5,100000.0,5000.0,100000.0,100000.0,1K,25.65,新宿区,改代町,東京メトロ有楽町線,江戸川橋駅,3
2,ステージグランデ神楽坂,賃貸マンション,12,8,7,110000.0,5000.0,110000.0,110000.0,1K,25.65,新宿区,改代町,東京メトロ有楽町線,江戸川橋駅,3
3,ステージグランデ神楽坂,賃貸マンション,12,8,8,145000.0,10000.0,145000.0,145000.0,1LDK,41.08,新宿区,改代町,東京メトロ有楽町線,江戸川橋駅,3
4,ステージグランデ神楽坂,賃貸マンション,12,8,1,150000.0,10000.0,0.0,0.0,1SK,41.86,新宿区,改代町,東京メトロ有楽町線,江戸川橋駅,3


In [36]:
df_new[df_new['最寄駅']=='青山一丁目駅']

,物件名,カテゴリー,築年数,構造,階数,賃料,管理費,敷金,礼金,間取り,面積,23区,都市名,路線,最寄駅,徒歩
4434,レジディア赤坂,賃貸マンション,19,5,3,159000.0,10000.0,159000.0,0.0,1DK,30.46,港区,赤坂７,東京メトロ半蔵門線,青山一丁目駅,10
4435,レジディア赤坂,賃貸マンション,19,5,3,199000.0,15000.0,199000.0,199000.0,1LDK,41.42,港区,赤坂７,東京メトロ半蔵門線,青山一丁目駅,10
4436,レジディア赤坂,賃貸マンション,19,5,4,248000.0,15000.0,248000.0,248000.0,2LDK,50.00,港区,赤坂７,東京メトロ半蔵門線,青山一丁目駅,10
4437,レジディア赤坂,賃貸マンション,19,5,5,485000.0,15000.0,485000.0,485000.0,3LDK,85.69,港区,赤坂７,東京メトロ半蔵門線,青山一丁目駅,10
4438,レジディア赤坂,賃貸マンション,19,5,5,485000.0,15000.0,485000.0,485000.0,3LDK,85.69,港区,赤坂７,東京メトロ半蔵門線,青山一丁目駅,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57080,アクシア青山,賃貸マンション,21,21,4,190000.0,15000.0,190000.0,190000.0,1K,40.48,港区,赤坂８,東京メトロ銀座線,青山一丁目駅,2
57112,アクシア青山,賃貸マンション,21,21,4,190000.0,15000.0,190000.0,190000.0,1K,40.48,港区,赤坂８,東京メトロ銀座線,青山一丁目駅,2
57135,シャトレー青山,賃貸マンション,59,5,5,130000.0,10000.0,130000.0,130000.0,1LDK,32.15,港区,南青山２,東京メトロ半蔵門線,青山一丁目駅,2
57136,シャトレー青山,賃貸マンション,59,5,4,130000.0,10000.0,260000.0,130000.0,1DK,30.70,港区,南青山２,東京メトロ半蔵門線,青山一丁目駅,2


In [41]:
df_new[df_new['物件名'].str.contains('タワー') & df_new['最寄駅'].str.contains('青山')]

,物件名,カテゴリー,築年数,構造,階数,賃料,管理費,敷金,礼金,間取り,面積,23区,都市名,路線,最寄駅,徒歩
18739,パークアクシス青山一丁目タワー,賃貸マンション,16,46,11,292000.0,0.0,1168000.0,0.0,ワンルーム,41.37,港区,南青山１,東京メトロ銀座線,青山一丁目駅,1
18740,パークアクシス青山一丁目タワー,賃貸マンション,16,46,14,292000.0,0.0,1168000.0,0.0,ワンルーム,41.37,港区,南青山１,東京メトロ銀座線,青山一丁目駅,1
18741,パークアクシス青山一丁目タワー,賃貸マンション,16,46,18,336000.0,0.0,1344000.0,0.0,ワンルーム,42.25,港区,南青山１,東京メトロ銀座線,青山一丁目駅,1
18742,パークアクシス青山一丁目タワー,賃貸マンション,16,46,11,520000.0,0.0,2080000.0,0.0,1LDK,71.65,港区,南青山１,東京メトロ銀座線,青山一丁目駅,1
23752,パークコート青山 ザ タワー,賃貸マンション,5,27,5,720000.0,30000.0,720000.0,720000.0,2LDK,80.91,港区,南青山２,東京メトロ銀座線,青山一丁目駅,4
23753,パークコート青山 ザ タワー,賃貸マンション,5,27,18,725000.0,30000.0,725000.0,1450000.0,2LDK,85.34,港区,南青山２,東京メトロ銀座線,青山一丁目駅,4
23754,パークコート青山 ザ タワー,賃貸マンション,5,27,15,780000.0,0.0,1560000.0,1560000.0,2LDK,80.57,港区,南青山２,東京メトロ銀座線,青山一丁目駅,4


In [42]:
df_new.to_csv('preprocessed_data_city.csv', index=None)

In [44]:
df_new[df_new['階数']==df_new['階数'].max()]

,物件名,カテゴリー,築年数,構造,階数,賃料,管理費,敷金,礼金,間取り,面積,23区,都市名,路線,最寄駅,徒歩
764,ザ・パークハウス西新宿タワー６０,賃貸マンション,6,60,57,520000.0,20000.0,520000.0,520000.0,2LDK,87.37,新宿区,西新宿５,都営大江戸線,西新宿五丁目駅,7
765,ザ・パークハウス西新宿タワー６０,賃貸マンション,6,60,57,520000.0,20000.0,520000.0,520000.0,2LDK,87.37,新宿区,西新宿５,都営大江戸線,西新宿五丁目駅,7


In [57]:
df_new[df_new['最寄駅'].str.contains('乃木坂')].sort_values(by='賃料', ascending=False).head(30)

,物件名,カテゴリー,築年数,構造,階数,賃料,管理費,敷金,礼金,間取り,面積,23区,都市名,路線,最寄駅,徒歩
6,テラス乃木坂,賃貸マンション,18,3,2,970000.0,0.0,970000.0,970000.0,4LDK,231.05,港区,赤坂９,東京メトロ千代田線,乃木坂駅,6
7,テラス乃木坂,賃貸マンション,18,3,2,970000.0,0.0,970000.0,970000.0,4LDK,231.05,港区,赤坂９,東京メトロ千代田線,乃木坂駅,6
678,パークコート赤坂檜町ザタワー,賃貸マンション,5,44,20,890000.0,0.0,1780000.0,890000.0,3LDK,86.59,港区,赤坂９,東京メトロ千代田線,乃木坂駅,3
677,パークコート赤坂檜町ザタワー,賃貸マンション,5,44,34,800000.0,0.0,1600000.0,1600000.0,2LDK,78.03,港区,赤坂９,東京メトロ千代田線,乃木坂駅,3
7179,東京メトロ千代田線 乃木坂駅 地下1地上44階建 築5年,賃貸マンション,5,44,34,800000.0,0.0,1600000.0,1600000.0,2LDK,78.30,港区,赤坂９,東京メトロ千代田線,乃木坂駅,3
24409,パークコート乃木坂ザ・タワー,賃貸マンション,4,25,18,750000.0,0.0,1500000.0,750000.0,2LDK,100.65,港区,南青山１,東京メトロ千代田線,乃木坂駅,3
7177,東京メトロ千代田線 乃木坂駅 地下1地上44階建 築5年,賃貸マンション,5,44,25,680000.0,20000.0,1360000.0,1360000.0,3SLDK,72.96,港区,赤坂９,東京メトロ千代田線,乃木坂駅,3
7176,東京メトロ千代田線 乃木坂駅 地下1地上44階建 築5年,賃貸マンション,5,44,25,680000.0,0.0,1360000.0,1360000.0,2SLDK,72.96,港区,赤坂９,東京メトロ千代田線,乃木坂駅,3
7175,東京メトロ千代田線 乃木坂駅 地下1地上44階建 築5年,賃貸マンション,5,44,35,680000.0,0.0,1360000.0,680000.0,3LDK,78.30,港区,赤坂９,東京メトロ千代田線,乃木坂駅,3
670,パークコート赤坂檜町ザタワー,賃貸マンション,5,44,35,680000.0,0.0,1360000.0,680000.0,3LDK,78.30,港区,赤坂９,東京メトロ千代田線,乃木坂駅,3
